In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np


In [2]:
class DCGAN():
    def __init__(self, rows, cols, channels, z = 10):
        # Input shape
        self.img_rows = rows
        self.img_cols = cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = z

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=256, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/dcgan_mnist_%d.png" % epoch)
        plt.close()


In [3]:
dcgan = DCGAN(28,28,1)
dcgan.train(epochs=5000, batch_size=256, save_interval=50)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_v2 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 8, 8, 64)          0

W0701 09:44:52.290865 140117499188992 training.py:2061] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
W0701 09:44:53.914172 140117499188992 training.py:2061] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?


0 [D loss: 1.032658, acc.: 37.89%] [G loss: 0.711535]
1 [D loss: 0.576665, acc.: 69.73%] [G loss: 0.979307]
2 [D loss: 0.303214, acc.: 90.82%] [G loss: 0.930681]
3 [D loss: 0.315156, acc.: 88.67%] [G loss: 0.627017]
4 [D loss: 0.489921, acc.: 74.41%] [G loss: 0.782907]
5 [D loss: 0.523139, acc.: 72.66%] [G loss: 1.326728]
6 [D loss: 0.475447, acc.: 77.54%] [G loss: 1.007548]
7 [D loss: 0.257883, acc.: 91.02%] [G loss: 0.325832]
8 [D loss: 0.176714, acc.: 97.46%] [G loss: 0.209266]
9 [D loss: 0.147136, acc.: 97.46%] [G loss: 0.216131]
10 [D loss: 0.242347, acc.: 91.41%] [G loss: 0.443748]
11 [D loss: 0.176513, acc.: 97.07%] [G loss: 0.604994]
12 [D loss: 0.137959, acc.: 98.05%] [G loss: 0.190660]
13 [D loss: 0.249862, acc.: 91.02%] [G loss: 0.144790]
14 [D loss: 0.489584, acc.: 75.39%] [G loss: 0.457229]
15 [D loss: 0.925210, acc.: 49.61%] [G loss: 1.062546]
16 [D loss: 1.012178, acc.: 41.60%] [G loss: 1.003114]
17 [D loss: 0.893808, acc.: 52.73%] [G loss: 1.039699]
18 [D loss: 0.727307

149 [D loss: 0.767170, acc.: 50.39%] [G loss: 1.201481]
150 [D loss: 0.805335, acc.: 51.95%] [G loss: 1.119935]
151 [D loss: 0.867119, acc.: 45.12%] [G loss: 1.051758]
152 [D loss: 0.851074, acc.: 44.53%] [G loss: 1.020815]
153 [D loss: 0.769382, acc.: 50.98%] [G loss: 1.098391]
154 [D loss: 0.833885, acc.: 48.05%] [G loss: 1.062298]
155 [D loss: 0.818894, acc.: 47.85%] [G loss: 1.083606]
156 [D loss: 0.852300, acc.: 42.77%] [G loss: 1.121276]
157 [D loss: 0.824623, acc.: 44.73%] [G loss: 1.137743]
158 [D loss: 0.769834, acc.: 50.20%] [G loss: 1.213203]
159 [D loss: 0.838310, acc.: 45.90%] [G loss: 1.072160]
160 [D loss: 0.806901, acc.: 45.31%] [G loss: 1.116753]
161 [D loss: 0.833816, acc.: 46.48%] [G loss: 1.047312]
162 [D loss: 0.815671, acc.: 47.27%] [G loss: 1.118469]
163 [D loss: 0.824194, acc.: 49.02%] [G loss: 1.074255]
164 [D loss: 0.870735, acc.: 44.34%] [G loss: 1.030170]
165 [D loss: 0.789821, acc.: 49.61%] [G loss: 1.104713]
166 [D loss: 0.815668, acc.: 45.70%] [G loss: 1.

296 [D loss: 0.826873, acc.: 45.70%] [G loss: 1.004067]
297 [D loss: 0.818482, acc.: 45.70%] [G loss: 0.957154]
298 [D loss: 0.849825, acc.: 42.19%] [G loss: 0.935838]
299 [D loss: 0.812425, acc.: 47.46%] [G loss: 0.956295]
300 [D loss: 0.823561, acc.: 44.34%] [G loss: 0.985807]
301 [D loss: 0.806507, acc.: 44.14%] [G loss: 0.916710]
302 [D loss: 0.841464, acc.: 44.14%] [G loss: 0.949240]
303 [D loss: 0.774449, acc.: 51.56%] [G loss: 1.010532]
304 [D loss: 0.789295, acc.: 46.09%] [G loss: 1.023168]
305 [D loss: 0.794232, acc.: 45.31%] [G loss: 1.015612]
306 [D loss: 0.778412, acc.: 47.46%] [G loss: 1.048830]
307 [D loss: 0.767364, acc.: 50.59%] [G loss: 0.982039]
308 [D loss: 0.745062, acc.: 52.93%] [G loss: 0.973208]
309 [D loss: 0.793812, acc.: 48.24%] [G loss: 1.054830]
310 [D loss: 0.719009, acc.: 54.49%] [G loss: 1.043904]
311 [D loss: 0.776557, acc.: 50.78%] [G loss: 1.047309]
312 [D loss: 0.776000, acc.: 47.46%] [G loss: 0.973092]
313 [D loss: 0.805089, acc.: 46.29%] [G loss: 0.

...

4346 [D loss: 0.719105, acc.: 48.63%] [G loss: 0.767247]
4347 [D loss: 0.707310, acc.: 48.44%] [G loss: 0.748032]
4348 [D loss: 0.706554, acc.: 50.00%] [G loss: 0.756917]
4349 [D loss: 0.708841, acc.: 49.22%] [G loss: 0.764245]
4350 [D loss: 0.711149, acc.: 46.09%] [G loss: 0.745145]
4351 [D loss: 0.710186, acc.: 50.78%] [G loss: 0.748824]
4352 [D loss: 0.708554, acc.: 47.66%] [G loss: 0.740376]
4353 [D loss: 0.711169, acc.: 47.07%] [G loss: 0.743909]
4354 [D loss: 0.699074, acc.: 51.76%] [G loss: 0.737529]
4355 [D loss: 0.719714, acc.: 45.70%] [G loss: 0.740169]
4356 [D loss: 0.713982, acc.: 45.90%] [G loss: 0.736274]
4357 [D loss: 0.712443, acc.: 46.68%] [G loss: 0.750034]
4358 [D loss: 0.720053, acc.: 44.92%] [G loss: 0.744473]
4359 [D loss: 0.719947, acc.: 43.55%] [G loss: 0.728273]
4360 [D loss: 0.707218, acc.: 48.63%] [G loss: 0.721638]
4361 [D loss: 0.708655, acc.: 49.80%] [G loss: 0.729490]
4362 [D loss: 0.708095, acc.: 48.44%] [G loss: 0.732305]
4363 [D loss: 0.711031, acc.: 4

4490 [D loss: 0.716958, acc.: 46.09%] [G loss: 0.744671]
4491 [D loss: 0.713685, acc.: 45.70%] [G loss: 0.745891]
4492 [D loss: 0.705711, acc.: 49.02%] [G loss: 0.745164]
4493 [D loss: 0.728993, acc.: 43.95%] [G loss: 0.740924]
4494 [D loss: 0.713875, acc.: 45.90%] [G loss: 0.729579]
4495 [D loss: 0.710605, acc.: 45.12%] [G loss: 0.750637]
4496 [D loss: 0.718853, acc.: 43.95%] [G loss: 0.721626]
4497 [D loss: 0.712784, acc.: 44.53%] [G loss: 0.750497]
4498 [D loss: 0.700164, acc.: 50.00%] [G loss: 0.741415]
4499 [D loss: 0.708688, acc.: 47.66%] [G loss: 0.729473]
4500 [D loss: 0.712008, acc.: 44.92%] [G loss: 0.746422]
4501 [D loss: 0.711878, acc.: 47.66%] [G loss: 0.750977]
4502 [D loss: 0.721172, acc.: 43.95%] [G loss: 0.734476]
4503 [D loss: 0.705192, acc.: 49.41%] [G loss: 0.744235]
4504 [D loss: 0.719530, acc.: 45.90%] [G loss: 0.730736]
4505 [D loss: 0.724532, acc.: 43.16%] [G loss: 0.718382]
4506 [D loss: 0.710110, acc.: 49.02%] [G loss: 0.726021]
4507 [D loss: 0.716737, acc.: 4

4634 [D loss: 0.709473, acc.: 47.07%] [G loss: 0.763231]
4635 [D loss: 0.717336, acc.: 42.97%] [G loss: 0.744887]
4636 [D loss: 0.699731, acc.: 48.83%] [G loss: 0.743211]
4637 [D loss: 0.705276, acc.: 49.41%] [G loss: 0.739674]
4638 [D loss: 0.715222, acc.: 46.88%] [G loss: 0.757884]
4639 [D loss: 0.712881, acc.: 46.68%] [G loss: 0.751016]
4640 [D loss: 0.711676, acc.: 44.53%] [G loss: 0.742426]
4641 [D loss: 0.715188, acc.: 46.68%] [G loss: 0.740926]
4642 [D loss: 0.714595, acc.: 45.51%] [G loss: 0.735886]
4643 [D loss: 0.721744, acc.: 43.16%] [G loss: 0.737486]
4644 [D loss: 0.706572, acc.: 47.85%] [G loss: 0.735393]
4645 [D loss: 0.719100, acc.: 45.12%] [G loss: 0.719942]
4646 [D loss: 0.716739, acc.: 44.53%] [G loss: 0.743348]
4647 [D loss: 0.714382, acc.: 46.68%] [G loss: 0.731252]
4648 [D loss: 0.708560, acc.: 46.09%] [G loss: 0.714576]
4649 [D loss: 0.704074, acc.: 49.41%] [G loss: 0.738955]
4650 [D loss: 0.710023, acc.: 46.09%] [G loss: 0.750095]
4651 [D loss: 0.711896, acc.: 4

4778 [D loss: 0.716452, acc.: 42.38%] [G loss: 0.745524]
4779 [D loss: 0.714932, acc.: 44.53%] [G loss: 0.757201]
4780 [D loss: 0.712851, acc.: 46.88%] [G loss: 0.737050]
4781 [D loss: 0.696655, acc.: 54.30%] [G loss: 0.768814]
4782 [D loss: 0.708829, acc.: 47.66%] [G loss: 0.735248]
4783 [D loss: 0.696459, acc.: 50.59%] [G loss: 0.772722]
4784 [D loss: 0.707366, acc.: 47.66%] [G loss: 0.739990]
4785 [D loss: 0.696558, acc.: 49.80%] [G loss: 0.761585]
4786 [D loss: 0.707353, acc.: 48.44%] [G loss: 0.746714]
4787 [D loss: 0.706014, acc.: 48.05%] [G loss: 0.734247]
4788 [D loss: 0.706514, acc.: 50.39%] [G loss: 0.734396]
4789 [D loss: 0.718607, acc.: 42.19%] [G loss: 0.739929]
4790 [D loss: 0.713296, acc.: 47.07%] [G loss: 0.754870]
4791 [D loss: 0.713139, acc.: 46.48%] [G loss: 0.755122]
4792 [D loss: 0.717235, acc.: 42.97%] [G loss: 0.746172]
4793 [D loss: 0.715955, acc.: 47.07%] [G loss: 0.734602]
4794 [D loss: 0.710962, acc.: 46.88%] [G loss: 0.726641]
4795 [D loss: 0.717083, acc.: 4

4922 [D loss: 0.720632, acc.: 45.12%] [G loss: 0.742207]
4923 [D loss: 0.719683, acc.: 42.58%] [G loss: 0.739047]
4924 [D loss: 0.707552, acc.: 46.29%] [G loss: 0.744616]
4925 [D loss: 0.705003, acc.: 47.85%] [G loss: 0.730388]
4926 [D loss: 0.708323, acc.: 46.48%] [G loss: 0.748181]
4927 [D loss: 0.718084, acc.: 44.92%] [G loss: 0.737757]
4928 [D loss: 0.719513, acc.: 41.80%] [G loss: 0.727306]
4929 [D loss: 0.704694, acc.: 46.29%] [G loss: 0.757129]
4930 [D loss: 0.710369, acc.: 45.31%] [G loss: 0.748853]
4931 [D loss: 0.717489, acc.: 44.92%] [G loss: 0.752832]
4932 [D loss: 0.712276, acc.: 45.12%] [G loss: 0.758878]
4933 [D loss: 0.706575, acc.: 49.22%] [G loss: 0.747037]
4934 [D loss: 0.705145, acc.: 48.63%] [G loss: 0.738912]
4935 [D loss: 0.710364, acc.: 46.68%] [G loss: 0.768676]
4936 [D loss: 0.704500, acc.: 47.46%] [G loss: 0.767500]
4937 [D loss: 0.701589, acc.: 49.02%] [G loss: 0.753923]
4938 [D loss: 0.703331, acc.: 50.78%] [G loss: 0.758575]
4939 [D loss: 0.713087, acc.: 4